In [1]:
from __future__ import print_function, division
import SimpleITK as sitk
import numpy as np
import csv
from glob import glob
import pandas as pd
import os

In [2]:
# List of image files
luna_path = 'D:/Data/LUNA16/'
luna_subset_path = luna_path+"subset0/"
output_path = "D:/Data/LUNA16/output/"
file_list=glob(luna_subset_path+"*.mhd")

def get_filename(file_list, case):
    for f in file_list:
        if case in f:
            return(f)

df_node = pd.read_csv(luna_path+"annotations.csv")
df_node["file"] = df_node["seriesuid"].map(lambda file_name: get_filename(file_list, file_name))
df_node = df_node.dropna()

In [23]:
def getframe(v_center):
    dimx1, dimx2, dimy1, dimy2 = 0,0,0,0
    if (v_center[0]-20)<0:
        dimx1, dimx2 = 0, 40
    elif (v_center[0]+20)>img_array.shape[0]:
        dimx1, dimx2 = img_array.shape[0]-40, img_array.shape[0] 
    else: 
        dimx1, dimx2 = v_center[0]-20, v_center[0]+20

    if (v_center[1]-20)<0:
        dimy1, dimy2 = 0, 40
    elif (v_center[1]+20)>img_array.shape[1]:
        dimy1, dimy2 = img_array.shape[1]-40, img_array.shape[1]
    else: 
        dimy1, dimy2 = v_center[1]-20, v_center[1]+20
    
    return dimx1,dimx2,dimy1,dimy2

def getnoduleinfo(cur_row):
    node_x = cur_row["coordX"]
    node_y = cur_row["coordY"]
    node_z = cur_row["coordZ"]
    
    center = np.array([node_x, node_y, node_z])
    origin = np.array(itk_img.GetOrigin())      # x,y,x in mm real world location? what point does this correspond to in the CT????
    spacing = np.array(itk_img.GetSpacing())
    v_center = np.rint((center-origin)/spacing)
    
    return center,origin,spacing,v_center

nodulecenterlist = list()

for filename in file_list:
    mini_df = df_node[df_node["file"]==filename]
    if mini_df.shape[0]>0:
        itk_img = sitk.ReadImage(filename) 
        img_array = sitk.GetArrayFromImage(itk_img)
        img_array = np.swapaxes(img_array,0,2)
        
        for node_idx, cur_row in mini_df.iterrows():       
            center,origin,spacing,v_center = getnoduleinfo(cur_row)
            nodulecenterlist.append((filename),v_center)
            
            
            #frame = np.zeros((40,40,3))
            #dimx1,dimx2,dimy1,dimy2 = getframe(v_center)
            #frame = img_array[dimx1:dimx2,dimy1:dimy2,v_center[2]-1:v_center[2]+2]
            #nodulearrays.append(frame)


C:\Users\Phantom77anto\Anaconda2\lib\site-packages\ipykernel\__main__.py:44: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


KeyboardInterrupt: 